In [7]:
import os
import tensorflow as tf
from pydub import AudioSegment
import numpy as np
import pandas as pd
from tabulate import tabulate

# Definir caminho para a pasta com os modelos
model_path = 'modelos'
audio_path ='audio'

# Obter lista de arquivos na pasta
files = os.listdir(model_path)
audios =os.listdir(audio_path)

# Filtrar apenas arquivos com extensão .h5
models = [f for f in files if f.endswith('.h5')]
audios = [a for a in audios if a.endswith('.wav')]

results=[]

# Loop através de todos os modelos na lista
for model_file in models:
    # Carregar o modelo
    model = tf.keras.models.load_model(os.path.join(model_path, model_file))
    
    # Preparar o áudio de entrada para o formato correto e normalizar os valores
    for audio_file in audios:
        audio_path = os.path.join(audio_path, audio_file)
        audio = AudioSegment.from_wav(audio_path)
        # converter amostragem para 22kHz e 16 bits
        converted_audio = normalized_audio.set_frame_rate(22050)
        converted_audio=converted_audio.set_channels(1)

        # normalizar os valores entre -1 e 1
        normalized_audio = converted_audio.normalize(headroom=0.1)

        audio_array=np.array(normalized_audio.get_array_of_samples())
        audio_array=audio_array.reshape(-1,1)

        # criar novo array com formato correto
        new_audio_array = np.zeros((1, 1, 22050, 1))

        # copiar valores do array original
        new_audio_array[0, 0, :len(audio_array), 0] = audio_array.flatten()

        # fazer a predição
        prediction = model.predict(new_audio_array)

        # armazenar os resultados em um dicionário
        result_dict = {
        'Model': os.path.basename(model_path),
        'Audio': os.path.basename(audio_path),
        'Prediction': 'Parkinson' if prediction > 0.5 else 'Não Parkinson',
        'Probability': prediction.item()
        }
        # adicionar o dicionário na lista de resultados
        results.append(result_dict)

# criar DataFrame com os resultados
results_df = pd.DataFrame(results)

# exibir a tabela em LaTeX
print(tabulate(results_df, headers='keys', tablefmt='latex_booktabs', showindex=False))


ValueError: could not broadcast input array from shape (1235968,) into shape (22050,)

In [ ]:
import os
import librosa
import pandas as pd

def parkinson_audio(audio_path):

    # Crie um DataFrame vazio
    df_audio = pd.DataFrame(columns=['name', 'path', 'meanfreq', 'sd', 'median', 'Q25', 'Q75', 'iqr', 'skew', 'kurt', 'sp.ent', 'sfm', 'mode', 'centroid', 'peakf', 'meanfun', 'minfun', 'maxfun', 'meandom', 'mindom', 'maxdom', 'dfrange', 'modindx', 'label'])

    # Preencha o DataFrame com as informações dos arquivos de áudio
    for audio_file in audio_files:
        name = os.path.splitext(audio_file)[0]
        path = os.path.join(audio_path, audio_file)
        label = 0  # ou 1, dependendo do áudio
        y, sr = librosa.load(path, sr=None)  # carregar o áudio com a taxa de amostragem original
        meanfreq = librosa.feature.spectral_centroid(y=y, sr=sr).mean()
        sd = librosa.feature.spectral_contrast(y=y, sr=sr).std()
        median = librosa.feature.spectral_rolloff(y=y, sr=sr).median()
        Q25 = librosa.feature.spectral_bandwidth(y=y, sr=sr).mean(axis=1)[0]
        Q75 = librosa.feature.spectral_bandwidth(y=y, sr=sr).mean(axis=1)[1]
        iqr = Q75 - Q25
        skew = librosa.feature.spectral_contrast(y=y, sr=sr).skew()
        kurt = librosa.feature.spectral_contrast(y=y, sr=sr).kurtosis()
        sp_ent = librosa.feature.spectral_contrast(y=y, sr=sr).mean(axis=1)[2]
        sfm = librosa.feature.spectral_flatness(y=y).mean()
        mode = librosa.feature.mode(y=y)[0][0]
        centroid = librosa.feature.spectral_centroid(y=y, sr=sr).mean()
        peakf = librosa.peak_pick(y=y, sr=sr, threshold=0.05, pre_max=3, post_max=3, pre_avg=3, post_avg=3, num_peaks=1)[0]
        meanfun = librosa.effects.pitch(y=y, sr=sr)[0]
        minfun = y.min()
        maxfun = y.max()
        meandom = librosa.feature.mfcc(y=y, sr=sr).mean()
        mindom = librosa.feature.mfcc(y=y, sr=sr).min()
        maxdom = librosa.feature.mfcc(y=y, sr=sr).max()
        dfrange = maxdom - mindom
        modindx = librosa.feature.tonnetz(y=y, sr=sr).mean()
        df_audio = df_audio.append({
            'name': name,
            'path': path,
            'meanfreq': meanfreq
    return df_audio
